<a href="https://colab.research.google.com/github/tlerksuthirat/lab_management/blob/main/CHEW_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from google.colab import auth  # Import 'auth' for authentication
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
import io
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Specify your Google Sheet ID
file_id = 'your_file_ID'

In [ ]:
# Authenticate and build the Google Drive API service
auth.authenticate_user()

# Build the Google Drive API service
drive_service = build('drive', 'v3')

In [ ]:
# Request to export the Google Sheet as a CSV
request = drive_service.files().export_media(fileId=file_id,mimeType='text/csv')

In [ ]:
# Handle file download
fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()

# Read data into a Dataframe
fh.seek(0)
df = pd.read_csv(fh)

In [ ]:
df.head(1)

In [ ]:
df.columns

In [ ]:
import datetime

# Convert the 'date' column to datetime format
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Select dates before 4/1/2024
filtered_df = df[df['Timestamp'] < pd.to_datetime('4/1/2024')]

In [ ]:
# Get the unique values in the "รายชื่อผู้เข้าร่วมโครงการ งานห้องปฏิบัติการวิจัย" column
unique_values = df["รายชื่อผู้เข้าร่วมโครงการ งานห้องปฏิบัติการวิจัย"].unique()

In [ ]:
unique_email = df["Email Address"].unique()
unique_email

In [ ]:
# Count unique emails
unique_email_count = len(df["Email Address"].unique())

# Print the result
print("Number of unique emails:", unique_email_count)

In [ ]:
# Count unique name
unique_name_count = len(df["รายชื่อผู้เข้าร่วมโครงการ งานห้องปฏิบัติการวิจัย"].unique())

# Print the result
print("Number of unique name:", unique_name_count)

In [ ]:
# Iterate through each unique value and filter the DataFrame
for name in unique_values:
    filtered_df = df[df['รายชื่อผู้เข้าร่วมโครงการ งานห้องปฏิบัติการวิจัย'] == name]

    # Export the filtered DataFrame to a CSV file
    file_name = f"{name}.csv"  # Create a filename based on the unique value
    filtered_df2.to_excel(file_name, index=False, engine='openpyxl')

In [ ]:
import zipfile

# Create a list of the filenames that you want to zip
filenames = [f"{name}.csv" for name in unique_values]

# Create a ZIP file object
with zipfile.ZipFile("Activity.zip", "w") as zip_file:

    # Add each file to the ZIP file
    for filename in filenames:
        zip_file.write(filename)

# Close the ZIP file
zip_file.close()

In [ ]:
def find_most_active(df):
    # Extract date only, discarding time components
    df['date'] = df['Timestamp'].dt.date

    # Count unique occurrences per name and date
    daily_counts = df.groupby(['รายชื่อผู้เข้าร่วมโครงการ งานห้องปฏิบัติการวิจัย', 'date'])['Timestamp'].count().reset_index(name ='count')

    # Find those with maximum registrations across days
    return daily_counts[daily_counts['count'] == daily_counts.groupby('รายชื่อผู้เข้าร่วมโครงการ งานห้องปฏิบัติการวิจัย')['count'].transform(max)]

def count_registration_days(df):
    # Extract date only
    df['date'] = df['Timestamp'].dt.date

    # Count the number of unique dates for each name
    registration_days = df.groupby('รายชื่อผู้เข้าร่วมโครงการ งานห้องปฏิบัติการวิจัย')['date'].nunique().reset_index(name='count')

     # Sort by count in descending order
    return registration_days.sort_values(by='count', ascending=False)


In [ ]:
print("Person(s) with most registrations:")
print(find_most_active(df.copy()))

print("\nRegistration days per person:")
print(count_registration_days(df.copy()))

In [ ]:
registration_counts_df = count_registration_days(df.copy())
registration_counts_df.to_excel('most_active_person.xlsx', index=False)